# This notebook shows example to load the CHGNet for prediction


In [ ]:
import numpy as np
import torch
from pymatgen.core import Structure

from chgnet.model import CHGNet

# If the above line fails in Google Colab due to numpy version issue,
# please restart the runtime, and the problem will be solved

np.set_printoptions(precision=4, suppress=True)

### Override torch.bincount (not supported by ONNX)

In [ ]:
def new_bincount(input_tensor, minlength: int = 0):
    counts = torch.tensor([], dtype=torch.int64)
    output_length = max(minlength, int(torch.max(input_tensor)) + 1)
    for value in range(output_length):
        value_count = (input_tensor == value).sum()
        new_count_tensor = torch.tensor([value_count])
        counts = torch.cat([counts, new_count_tensor])
    return counts

torch.bincount=new_bincount

### Read structure from a json or cif file


In [ ]:
structure = Structure.from_file("/Users/chrisfajardo/git_repos/pochi/Li.cif")

### Load Model


In [ ]:
chgnet = CHGNet.load()

# Alternatively you can read your own model
# chgnet = CHGNet.from_file(model_path)

In [ ]:
graph = chgnet.graph_converter(structure)

In [ ]:
chgnet = chgnet.to("cpu")

In [ ]:
chgnet = chgnet.eval()

In [ ]:
chgnet.forward([list(graph)])

### Create traced model and save TorchScript

In [ ]:
traced = torch.jit.trace(chgnet, ([list(graph)],))

In [ ]:
traced.save("chgnet.torchscript")

In [ ]:
c = torch.jit.load("chgnet.torchscript")

In [ ]:
c.forward([list(graph)])

### Try to export to ONNX

In [14]:
torch.onnx.export(traced, [list(graph)], f="chgnet.onnx")

SymbolicValueError: Cannot determine scalar type for this '<class 'torch.TensorType'>' instance and a default value was not provided.  [Caused by the value '182 defined in (%182 : Tensor = onnx::Gather[axis=1](%157, %149), scope: chgnet.model.model.CHGNet:: # /Users/chrisfajardo/git_repos/chgnet/chgnet/model/model.py:875:0
)' (type 'Tensor') in the TorchScript graph. The containing node has kind 'onnx::Gather'.] 
    (node defined in /Users/chrisfajardo/git_repos/chgnet/chgnet/model/model.py(875): get_batch_graph_from_graphs
/Users/chrisfajardo/git_repos/chgnet/chgnet/model/model.py(382): forward
/Users/chrisfajardo/miniconda3/envs/chgnet/lib/python3.9/site-packages/torch/nn/modules/module.py(1522): _slow_forward
/Users/chrisfajardo/miniconda3/envs/chgnet/lib/python3.9/site-packages/torch/nn/modules/module.py(1541): _call_impl
/Users/chrisfajardo/miniconda3/envs/chgnet/lib/python3.9/site-packages/torch/nn/modules/module.py(1532): _wrapped_call_impl
/Users/chrisfajardo/miniconda3/envs/c

In [ ]:
#scripted = torch.jit.script(chgnet, example_inputs=[([graph.to_dict()],)])